## Решение задачи:

Импортируем необходимые библиотеки и отключим вывод ошибок:

In [ ]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # Отключает большинство предупреждений
import tensorflow as tf
tf.get_logger().setLevel('ERROR')  # Дополнительное отключение логов

import random
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from keras.models import Sequential
from keras.layers import Dense, Input, Conv1D, MaxPooling1D, Flatten, SimpleRNN, LSTM
from keras.optimizers import Adam

import warnings
warnings.filterwarnings('ignore')

Выполним загрузку данных и воспользуемся функцией `train_test_split` для разделения данных на обучающую и тестовые выборки. Обучающая выборка получает 80% данных, а тестовая 20%:

In [ ]:
# Загрузка данных
data = load_iris()  # загружаем набор данных "Ирисы Фишера"
X = data.data       # матрица признаков
y = data.target     # вектор целевой переменной

# Разделение данных на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)  # разделяем данные на обучающую и тестовую выборки (20% тестовых данных)

Создается объект `StandardScaler` для стандартизации данных, который настраивается на обучающей выборке и затем применяется к тестовой выборке. Это обеспечивает одинаковое масштабирование признаков в обеих выборках, что критично для корректной работы нейронных сетей и сравнения результатов:

In [ ]:
# Нормализация данных
scaler = StandardScaler()  # создаем объект для стандартизации данных
X_train = scaler.fit_transform(X_train)  # нормализуем обучающую выборку
X_test = scaler.transform(X_test)  # нормализуем тестовую выборку

Данные преобразуются в трехмерный формат для **CNN** и **RNN**, поскольку эти архитектуры требуют разной организации входных данных. Для CNN создается форма [сэмплы, признаки, каналы], а для RNN - [сэмплы, временные шаги, признаки], что соответствует ожидаемой структуре входов для сверточных и рекуррентных слоев соответственно.

**Сэмплы** - это отдельные объекты данных (цветки ириса), каждый со своими измерениями.

**Временные шаги** - это условные моменты "времени" в последовательности, где на каждом шаге присутствует набор признаков цветка.

In [ ]:
# Для CNN и RNN нужно изменить форму данных
X_train_cnn = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
X_test_cnn = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)
X_train_rnn = X_train.reshape(X_train.shape[0], 1, X_train.shape[1])
X_test_rnn = X_test.reshape(X_test.shape[0], 1, X_test.shape[1])

Эта функция создает, обучает и оценивает нейронную сеть на основе переданных параметров. Возвращает точность модели на тестовой выборке как меру приспособленности особи:

In [ ]:
def fitness_function(params):
    """
    Оценка качества особи.

    Args:
        params: кортеж параметров (model_type, learning_rate, layers, neurons)
            - model_type: тип модели ('MLP', 'CNN', 'RNN')
            - learning_rate: скорость обучения (0.001-0.1)
            - layers: количество слоев (1-5) - интерпретация зависит от типа модели
            - neurons: количество нейронов/фильтров (10-100)

    Returns:
        accuracy: точность модели на тестовой выборке (0-1)
    """
    model_type, learning_rate, layers, neurons = params

    if model_type == 'MLP':
        model = Sequential()
        model.add(Input(shape=(X_train.shape[1],))) # Входной слой
        for _ in range(layers):
            model.add(Dense(neurons, activation='relu'))  # Полносвязный слой с ReLU
        model.add(Dense(3, activation='softmax')) # Выходной слой

        X_train_used, X_test_used = X_train, X_test

    elif model_type == 'CNN':
        model = Sequential()
        model.add(Input(shape=(X_train.shape[1], 1))) # Входной слой
        model.add(Conv1D(neurons, 3, activation='relu', padding='same'))
        model.add(MaxPooling1D(pool_size=2))
        model.add(Flatten())
        model.add(Dense(neurons, activation='relu'))
        model.add(Dense(3, activation='softmax')) # Выходной слой

        X_train_used, X_test_used = X_train_cnn, X_test_cnn

    elif model_type == 'RNN':
        model = Sequential()
        model.add(Input(shape=(1, X_train.shape[1]))) # Входной слой
        model.add(SimpleRNN(neurons, activation='relu'))
        model.add(Dense(3, activation='softmax')) # Выходной слой

        X_train_used, X_test_used = X_train_rnn, X_test_rnn

    # Компиляция модели
    model.compile(optimizer=Adam(learning_rate=learning_rate),
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

    model.fit(X_train_used, y_train, epochs=5, verbose=0) # Обучение модели

    # Предсказание на тестовой выборке
    y_pred_probs = model.predict(X_test_used, verbose=0)

    # Преобразование вероятностей в предсказанные классы
    y_pred = np.argmax(y_pred_probs, axis=1)

    # Вычисление точности предсказаний
    accuracy = accuracy_score(y_test, y_pred)

    return accuracy # Возвращаем точность как меру приспособленности

Алгоритм имитирует естественный отбор для поиска оптимальных гиперпараметров через последовательность поколений с селекцией, кроссовером и мутацией:

In [ ]:
def genetic_algorithm(population_size, num_generations, mutation_rate):
    """
    Реализация генетического алгоритма

    Args:
        population_size: количество особей в популяции
        num_generations: количество поколений эволюции
        mutation_rate: вероятность мутации особи (0-1)

    Returns:
        best_individual: лучшая найденная особь (параметры)
        best_fitness: точность лучшей особи
        model_type_history: история распределения типов моделей по поколениям
    """
    population = []  # Список для хранения популяции особей

    # Создание начальной популяции со случайными параметрами
    for _ in range(population_size):
        model_type = random.choice(['MLP', 'CNN', 'RNN'])         # Случайный тип модели
        learning_rate = random.uniform(0.001, 0.1)                # Случайная скорость обучения
        layers = random.randint(1, 5)                             # Случайное количество слоев
        neurons = random.randint(10, 100)                         # Случайное количество нейронов

        # Добавляем особь в популяцию
        population.append((model_type, learning_rate, layers, neurons))

    # Инициализация переменных для отслеживания лучшего результата
    best_fitness = 0           # Лучшая точность за все поколения
    best_individual = None     # Лучшая особь за все поколения
    model_type_history = []    # История для сбора статистики по типам моделей

    for generation in range(num_generations):
        print(f"\nПоколение {generation + 1}/{num_generations}")

        fitness_scores = []              # Список для хранения точностей особей
        generation_model_types = []      # Список для типов моделей текущего поколения

        # Оценка каждой особи в популяции
        for idx, individual in enumerate(population):
            # Вычисление приспособленности (точности) особи
            fitness = fitness_function(individual)
            fitness_scores.append(fitness)

            # Сохраняем тип модели для статистики
            generation_model_types.append(individual[0])

            # Детальный вывод информации об особи
            model_type, lr, layers, neurons = individual
            param_desc = f"LR={lr:.4f}, Слои={layers}, Нейроны={neurons}"
            print(f"Особь {idx + 1}: {model_type}, {param_desc}, Точность={fitness:.4f}")

        # Сохраняем распределение моделей текущего поколения для анализа
        model_type_history.append(generation_model_types)

        current_best_idx = np.argmax(fitness_scores)                    # Индекс лучшей особи
        current_best_fitness = fitness_scores[current_best_idx]         # Лучшая точность поколения
        current_best_individual = population[current_best_idx]          # Лучшая особь поколения

        if current_best_fitness > best_fitness:
            best_fitness = current_best_fitness
            best_individual = current_best_individual

            # Вывод информации о новом лучшем результате
            model_type, lr, layers, neurons = best_individual
            param_desc = f"LR={lr:.4f}, Слои={layers}, Нейроны={neurons}"
            print(f"\nНовый лучший результат!")
            print(f"Модель: {model_type}, Параметры: {param_desc}")
            print(f"Точность: {best_fitness:.4f}")

        selected_population = []  # Особи, отобранные для размножения

        for _ in range(population_size):
            # Турнир из 3 случайных особей
            tournament = random.sample(list(zip(population, fitness_scores)), 3)

            # Выбор победителя турнира (особь с максимальной точностью)
            winner = max(tournament, key=lambda x: x[1])[0]

            # Добавление победителя в список отобранных
            selected_population.append(winner)

        new_population = []  # Новое поколение особей

        # Обработка пар родителей (шаг 2 для создания пар потомков)
        for i in range(0, population_size, 2):
            parent1, parent2 = selected_population[i], selected_population[i + 1]

            # Кроссовер: создание двух потомков от пары родителей
            child1, child2 = crossover(parent1, parent2)

            # Мутация: случайное изменение потомков с заданной вероятностью
            child1 = mutate(child1, mutation_rate)
            child2 = mutate(child2, mutation_rate)

            # Добавление потомков в новую популяцию
            new_population.append(child1)
            new_population.append(child2)

        # Замена старой популяции новой
        population = new_population

    # Возврат лучшего результата и истории для анализа
    return best_individual, best_fitness, model_type_history

Функция `crossover` cоздает двух потомков путем комбинации параметров двух родителей. Используется одноточечный кроссовер:
    

In [ ]:
def crossover(parent1, parent2):
    """
    Скрещивание особей

    Args:
        parent1: первый родитель (model_type, lr, layers, neurons)
        parent2: второй родитель (model_type, lr, layers, neurons)

    Returns:
        child1, child2: два потомка с комбинированными параметрами
    """
    # Случайный выбор точки кроссовера (1-3)
    crossover_point = random.randint(1, 3)

    # Создание потомков:
    # - child1: первые crossover_point параметров от parent1, остальные от parent2
    # - child2: первые crossover_point параметров от parent2, остальные от parent1
    child1 = parent1[:crossover_point] + parent2[crossover_point:]
    child2 = parent2[:crossover_point] + parent1[crossover_point:]

    return child1, child2

Функция `mutate` cлучайным образом изменяет параметры особи с заданной вероятностью. При мутации все параметры особи генерируются заново:

In [ ]:
def mutate(individual, mutation_rate):
    """
    Мутирование особей

    Args:
        individual: особь для возможной мутации
        mutation_rate: вероятность мутации (0-1)

    Returns:
        individual: исходная или мутировавшая особь
    """
    # Проверка вероятности мутации
    if random.random() < mutation_rate:
        # Разбор параметров особи
        model_type, learning_rate, layers, neurons = individual

        # Полная замена всех параметров случайными значениями:
        learning_rate = random.uniform(0.001, 0.1)    # Новая скорость обучения
        layers = random.randint(1, 5)                 # Новое количество слоев
        neurons = random.randint(10, 100)             # Новое количество нейронов

        # Создание мутировавшей особи (тип модели сохраняется)
        individual = (model_type, learning_rate, layers, neurons)

    return individual

С помощью следующей функции построим круговую диаграмму, показывающую процентное соотношение разных типов моделей за все поколения генетического алгоритма:

In [ ]:
def plot_model_distribution(model_type_history):
    """
    Визуализируем распределение типов моделей

    Args:
        model_type_history: список списков с типами моделей каждого поколения
    """
    # Преобразование вложенного списка в плоский список всех моделей
    all_models = [model for generation in model_type_history for model in generation]

    # Подсчет количества каждого типа моделей
    model_counts = {
        'MLP': all_models.count('MLP'),
        'CNN': all_models.count('CNN'),
        'RNN': all_models.count('RNN')
    }

    # Создание круговой диаграммы с отверстием посередине
    fig = go.Figure(data=[go.Pie(
        labels=list(model_counts.keys()),      # Названия типов моделей
        values=list(model_counts.values()),    # Количество каждого типа
        hole=0.3,                              # Размер отверстия в центре (0.3 = 30%)
        marker_colors=['#FF6B6B', '#4ECDC4', '#45B7D1'],  # Цвета для каждого сегмента
        textinfo='label+percent+value',        # Отображаемая информация на диаграмме
        hovertemplate='<b>%{label}</b><br>Количество: %{value}<br>Процент: %{percent}'
    )])

    # Настройка внешнего вида диаграммы
    fig.update_layout(
        title='Распределение типов моделей в популяции',
        annotations=[dict(text='Модели', x=0.5, y=0.5, font_size=20, showarrow=False)]
    )

    # Отображение диаграммы
    fig.show()

    print("\nСТАТИСТИКА РАСПРЕДЕЛЕНИЯ МОДЕЛЕЙ")
    total_models = len(all_models)

    # Вывод статистики для каждого типа модели
    for model_type, count in model_counts.items():
        percentage = (count / total_models) * 100
        print(f"{model_type}: {count} особей ({percentage:.1f}%)")

Укажем гиперпараметры для обучения:

In [ ]:
population_size = 10  # размер популяции
num_generations = 10  # количество поколений
mutation_rate = 0.1  # вероятность мутации

Воспользуемся функцией `genetic_algorithm`, запустив генетический алгоритм:

In [ ]:
print("Запуск генетического алгоритма...")
best_params, best_accuracy, model_history = genetic_algorithm(population_size, num_generations, mutation_rate)

Запуск генетического алгоритма...

Поколение 1/10
Особь 1: RNN, LR=0.0757, Слои=2, Нейроны=58, Точность=0.9667
Особь 2: CNN, LR=0.0208, Слои=1, Нейроны=67, Точность=0.9667
Особь 3: RNN, LR=0.0063, Слои=2, Нейроны=10, Точность=0.8000
Особь 4: MLP, LR=0.0569, Слои=1, Нейроны=94, Точность=0.9667
Особь 5: CNN, LR=0.0918, Слои=5, Нейроны=61, Точность=1.0000
Особь 6: MLP, LR=0.0784, Слои=5, Нейроны=52, Точность=0.9333
Особь 7: CNN, LR=0.0668, Слои=3, Нейроны=60, Точность=1.0000
Особь 8: CNN, LR=0.0280, Слои=2, Нейроны=99, Точность=1.0000
Особь 9: RNN, LR=0.0566, Слои=3, Нейроны=55, Точность=0.9667
Особь 10: RNN, LR=0.0167, Слои=3, Нейроны=31, Точность=0.9667

Новый лучший результат!
Модель: CNN, Параметры: LR=0.0918, Слои=5, Нейроны=61
Точность: 1.0000

Поколение 2/10
Особь 1: CNN, LR=0.0668, Слои=3, Нейроны=60, Точность=0.9000
Особь 2: CNN, LR=0.0918, Слои=5, Нейроны=61, Точность=0.9667
Особь 3: CNN, LR=0.0918, Слои=5, Нейроны=61, Точность=0.9667
Особь 4: CNN, LR=0.0918, Слои=5, Нейроны=61,

Построим график и узнаем статистику распределения моделей:

In [ ]:
plot_model_distribution(model_history)


СТАТИСТИКА РАСПРЕДЕЛЕНИЯ МОДЕЛЕЙ
MLP: 2 особей (2.0%)
CNN: 91 особей (91.0%)
RNN: 7 особей (7.0%)


Выведем итоговые результаты обучения моделей, выявив лучшую из них:

In [ ]:
# Вывод итоговых результатов
print("\nИтоговые результаты:")
model_type, lr, layers, neurons = best_params
param_desc = f"LR={lr:.4f}, Слои={layers}, Нейроны={neurons}"
print(f"Лучшая модель: {model_type}")
print(f"Лучшие гиперпараметры: {param_desc}")
print(f"Лучшая точность: {best_accuracy:.4f}")


Итоговые результаты:
Лучшая модель: CNN
Лучшие гиперпараметры: LR=0.0918, Слои=5, Нейроны=61
Лучшая точность: 1.0000


## Вывод:

Генетический алгоритм корректно работает - он нашел, что **CNN** наиболее эффективна для нашей задачи и данных. Это доказательство того, что эволюционный процесс успешно исследует пространство моделей и находит оптимальные решения!

Статистика 91% **CNN** говорит о том, что для датасета `Iris` сверточные сети оказались значительно эффективнее других архитектур в условиях нашего эксперимента.